<a href="https://colab.research.google.com/github/kirath2205/Final-Year-Project/blob/main/FGSM_attack_and_PGD_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import keras,os
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D , Dropout
import numpy as np
from tensorflow import keras
from keras.initializers import glorot_uniform
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input
from keras import regularizers
from absl import app, flags

# Install bleeding edge version of cleverhans
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

  Cloning https://github.com/tensorflow/cleverhans.git to /tmp/pip-install-etj41kv8/cleverhans_70a0d60019d343e29fb08ed94cdd7c6a
  Running command git clone -q https://github.com/tensorflow/cleverhans.git /tmp/pip-install-etj41kv8/cleverhans_70a0d60019d343e29fb08ed94cdd7c6a


In [11]:
cnn_model = keras.models.load_model('/content/cnn_model_cifar10.h5')
vgg_16 = keras.models.load_model('/content/vgg16_cifar10.h5')
resnet = keras.models.load_model('/content/resnet_cifar10.h5')

In [12]:
cifar_10 = tf.keras.datasets.cifar10.load_data()
(X_train, y_train), (X_test, y_test) = cifar_10
def pre_processing_cifar10(X_train,y_train,X_test,y_test,image_channel):
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')

  y_train = keras.utils.to_categorical(y_train)
  y_test = keras.utils.to_categorical(y_test)

  X_train = preprocess_input(X_train)
  X_test = preprocess_input(X_test)

  img_width = X_train[0].shape[0]
  img_height = X_train[0].shape[1]
  image_channel = image_channel

  input_shape=(img_width,img_height,image_channel) 
  return X_train,y_train,X_test,y_test
X_train,y_train,X_test,y_test = pre_processing_cifar10(X_train,y_train,X_test,y_test,3)

In [13]:

def make_predictions(model , image , true_value):

  labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

  true_label_index = -1
  for i in range(len(true_value)):

    if(true_value[i]==1):
      true_label_index = i
      break

  prediction = model.predict(image)[0]
  probability = float('-inf')
  predicted_label_index = -1

  for i in range(len(prediction)):

    if(prediction[i]>probability):
      probability = prediction[i]
      predicted_label_index = i

  predicted_label = labels[predicted_label_index]
  true_label = labels[true_label_index]
  print()
  print()
  print('Predicted Label -',predicted_label,' Probability -',probability)
  print('True Label -',true_label)
  print()
  print()

  return predicted_label , probability 


#FGSM and PGD attacks on standard cnn model

In [14]:
from keras.preprocessing.image import img_to_array

epsilon = 0.2
logits_cnn_model = tf.keras.Model(cnn_model.input,cnn_model.layers[-1].output)
total_images = 200
misclassifications_fgsm = []
misclassifications_pgd = []
FLAGS = flags.FLAGS
print('Standard CNN result')
print()
for image_index in range(total_images):
  
  print('-------------------------------')
  print()
  print('Image index :',image_index)
  print()
  print('Original Sample')
  image  = X_test[image_index]
  image = img_to_array(image)
  image = image.reshape(1, 32, 32, 3)
  true_value = y_test[image_index]
  original_prediction = make_predictions(cnn_model,image,true_value)
  fgsm_sample = fast_gradient_method(logits_cnn_model, image, epsilon, np.inf, targeted=False)
  pgd_sample = projected_gradient_descent(cnn_model, image, 0.2, 0.01, 40, np.inf)
  print()
  print('Untargeted FGSM Adverserial Sample')
  fgsm_prediction = make_predictions(cnn_model , fgsm_sample , true_value)
  print()
  print('PGD Adverserial Sample')
  pgd_prediction = make_predictions(cnn_model , pgd_sample , true_value)
  print()
  
  if(pgd_prediction[0]!=original_prediction[0]):
    original_model_label = original_prediction[0]
    pgd_label = pgd_prediction[0]
    original_probability = original_prediction[1]
    pgd_probability = pgd_prediction[1]
    misclassifications_pgd.append({'Original Model Label':original_model_label,'original probability':original_probability,'Prediction after PGD attack':pgd_label,'Probability after PGD attack':pgd_probability,'Image index':image_index})

  if(fgsm_prediction[0]!=original_prediction[0]):
    original_model_label = original_prediction[0]
    fgsm_label = fgsm_prediction[0]
    original_probability = original_prediction[1]
    fgsm_probability = fgsm_prediction[1]
    misclassifications_fgsm.append({'Original Model Label':original_model_label,'original probability':original_probability,'Prediction after FGSM attack':fgsm_label,'Probability after FGSM attack':fgsm_probability,'Image index':image_index})
     
  print('-------------------------------')


Streaming output truncated to the last 5000 lines.



PGD Adverserial Sample


Predicted Label - airplane  Probability - 0.7013967
True Label - airplane



-------------------------------
-------------------------------

Image index : 28

Original Sample


Predicted Label - truck  Probability - 0.9926789
True Label - truck



Untargeted FGSM Adverserial Sample


Predicted Label - truck  Probability - 0.9826978
True Label - truck



PGD Adverserial Sample


Predicted Label - truck  Probability - 0.9824228
True Label - truck



-------------------------------
-------------------------------

Image index : 29

Original Sample


Predicted Label - frog  Probability - 0.9997577
True Label - frog



Untargeted FGSM Adverserial Sample


Predicted Label - frog  Probability - 0.99940324
True Label - frog



PGD Adverserial Sample


Predicted Label - frog  Probability - 0.999388
True Label - frog



-------------------------------
-------------------------------

Image index : 30

Original Sample

In [15]:
print('Mispredictions due to FGSM attack')
print()
print('-------------------------------')
counter = 0
for element in misclassifications_fgsm:
  print(element)
  counter+=1
print()
print('Total count - ',counter)

Mispredictions due to FGSM attack

-------------------------------
{'Original Model Label': 'deer', 'original probability': 0.49655074, 'Prediction after FGSM attack': 'frog', 'Probability after FGSM attack': 0.3304782, 'Image index': 12}
{'Original Model Label': 'frog', 'original probability': 0.4447179, 'Prediction after FGSM attack': 'bird', 'Probability after FGSM attack': 0.3325661, 'Image index': 25}
{'Original Model Label': 'deer', 'original probability': 0.40816933, 'Prediction after FGSM attack': 'dog', 'Probability after FGSM attack': 0.38430256, 'Image index': 26}
{'Original Model Label': 'cat', 'original probability': 0.3551306, 'Prediction after FGSM attack': 'bird', 'Probability after FGSM attack': 0.36996412, 'Image index': 33}
{'Original Model Label': 'ship', 'original probability': 0.48950055, 'Prediction after FGSM attack': 'cat', 'Probability after FGSM attack': 0.7988166, 'Image index': 35}
{'Original Model Label': 'deer', 'original probability': 0.6551083, 'Predict

In [16]:
print('Mispredictions due to PGD attack')
print()
print('-------------------------------')
counter = 0
for element in misclassifications_pgd:
  print(element)
  counter+=1
print()
print(counter)

Mispredictions due to PGD attack

-------------------------------
{'Original Model Label': 'deer', 'original probability': 0.49655074, 'Prediction after PGD attack': 'frog', 'Probability after PGD attack': 0.33988628, 'Image index': 12}
{'Original Model Label': 'frog', 'original probability': 0.4447179, 'Prediction after PGD attack': 'bird', 'Probability after PGD attack': 0.33433905, 'Image index': 25}
{'Original Model Label': 'deer', 'original probability': 0.40816933, 'Prediction after PGD attack': 'dog', 'Probability after PGD attack': 0.38330615, 'Image index': 26}
{'Original Model Label': 'cat', 'original probability': 0.3551306, 'Prediction after PGD attack': 'bird', 'Probability after PGD attack': 0.37812552, 'Image index': 33}
{'Original Model Label': 'ship', 'original probability': 0.48950055, 'Prediction after PGD attack': 'cat', 'Probability after PGD attack': 0.8077393, 'Image index': 35}
{'Original Model Label': 'deer', 'original probability': 0.6551083, 'Prediction after

#FGSM and PGD attacks on vgg16

In [17]:
from keras.preprocessing.image import img_to_array

epsilon = 0.2
logits_vgg_16 = tf.keras.Model(vgg_16.input,vgg_16.layers[-1].output)
total_images = 200
misclassifications_fgsm = []
misclassifications_pgd = []
FLAGS = flags.FLAGS
print('VGG16 result')
print()
for image_index in range(total_images):
  
  print('-------------------------------')
  print()
  print('Image index :',image_index)
  print()
  print('Original Sample')
  image  = X_test[image_index]
  image = img_to_array(image)
  image = image.reshape(1, 32, 32, 3)
  true_value = y_test[image_index]
  original_prediction = make_predictions(vgg_16,image,true_value)
  fgsm_sample = fast_gradient_method(logits_vgg_16, image, epsilon, np.inf, targeted=False)
  pgd_sample = projected_gradient_descent(vgg_16, image, 0.2, 0.01, 40, np.inf)
  print()
  print('Untargeted FGSM Adverserial Sample')
  fgsm_prediction = make_predictions(vgg_16 , fgsm_sample , true_value)
  print()
  print('PGD Adverserial Sample')
  pgd_prediction = make_predictions(vgg_16 , pgd_sample , true_value)
  print()
  
  if(pgd_prediction[0]!=original_prediction[0]):
    original_model_label = original_prediction[0]
    pgd_label = pgd_prediction[0]
    original_probability = original_prediction[1]
    pgd_probability = pgd_prediction[1]
    misclassifications_pgd.append({'Original Model Label':original_model_label,'original probability':original_probability,'Prediction after PGD attack':pgd_label,'Probability after PGD attack':pgd_probability,'Image index':image_index})

  if(fgsm_prediction[0]!=original_prediction[0]):
    original_model_label = original_prediction[0]
    fgsm_label = fgsm_prediction[0]
    original_probability = original_prediction[1]
    fgsm_probability = fgsm_prediction[1]
    misclassifications_fgsm.append({'Original Model Label':original_model_label,'original probability':original_probability,'Prediction after FGSM attack':fgsm_label,'Probability after FGSM attack':fgsm_probability,'Image index':image_index})
     
  print('-------------------------------')


Streaming output truncated to the last 5000 lines.



PGD Adverserial Sample


Predicted Label - airplane  Probability - 0.99942005
True Label - airplane



-------------------------------
-------------------------------

Image index : 28

Original Sample


Predicted Label - truck  Probability - 0.9997899
True Label - truck



Untargeted FGSM Adverserial Sample


Predicted Label - truck  Probability - 0.9997259
True Label - truck



PGD Adverserial Sample


Predicted Label - truck  Probability - 0.99972457
True Label - truck



-------------------------------
-------------------------------

Image index : 29

Original Sample


Predicted Label - frog  Probability - 0.9995127
True Label - frog



Untargeted FGSM Adverserial Sample


Predicted Label - frog  Probability - 0.9992137
True Label - frog



PGD Adverserial Sample


Predicted Label - frog  Probability - 0.99920493
True Label - frog



-------------------------------
-------------------------------

Image index : 30

Original Sam

In [18]:
print('Mispredictions due to FGSM attack')
print()
print('-------------------------------')
counter = 0
for element in misclassifications_fgsm:
  print(element)
  counter+=1
print()
print('Total count - ',counter)

Mispredictions due to FGSM attack

-------------------------------
{'Original Model Label': 'automobile', 'original probability': 0.651145, 'Prediction after FGSM attack': 'truck', 'Probability after FGSM attack': 0.5908351, 'Image index': 6}
{'Original Model Label': 'frog', 'original probability': 0.45619446, 'Prediction after FGSM attack': 'deer', 'Probability after FGSM attack': 0.86560816, 'Image index': 7}
{'Original Model Label': 'airplane', 'original probability': 0.63287616, 'Prediction after FGSM attack': 'deer', 'Probability after FGSM attack': 0.9076086, 'Image index': 22}
{'Original Model Label': 'deer', 'original probability': 0.6187001, 'Prediction after FGSM attack': 'cat', 'Probability after FGSM attack': 0.76818806, 'Image index': 26}
{'Original Model Label': 'dog', 'original probability': 0.365105, 'Prediction after FGSM attack': 'horse', 'Probability after FGSM attack': 0.39473933, 'Image index': 42}
{'Original Model Label': 'airplane', 'original probability': 0.4992

In [19]:
print('Mispredictions due to PGD attack')
print()
print('-------------------------------')
counter = 0
for element in misclassifications_pgd:
  print(element)
  counter+=1
print()
print(counter)

Mispredictions due to PGD attack

-------------------------------
{'Original Model Label': 'automobile', 'original probability': 0.651145, 'Prediction after PGD attack': 'truck', 'Probability after PGD attack': 0.5947029, 'Image index': 6}
{'Original Model Label': 'frog', 'original probability': 0.45619446, 'Prediction after PGD attack': 'deer', 'Probability after PGD attack': 0.8827358, 'Image index': 7}
{'Original Model Label': 'airplane', 'original probability': 0.63287616, 'Prediction after PGD attack': 'deer', 'Probability after PGD attack': 0.91916573, 'Image index': 22}
{'Original Model Label': 'deer', 'original probability': 0.6187001, 'Prediction after PGD attack': 'cat', 'Probability after PGD attack': 0.78513694, 'Image index': 26}
{'Original Model Label': 'bird', 'original probability': 0.6425198, 'Prediction after PGD attack': 'deer', 'Probability after PGD attack': 0.402931, 'Image index': 32}
{'Original Model Label': 'dog', 'original probability': 0.365105, 'Prediction a

#FGSM and PGD attacks on resnet model

In [20]:
from keras.preprocessing.image import img_to_array

epsilon = 0.2
logits_resnet = tf.keras.Model(resnet.input,resnet.layers[-1].output)
total_images = 200
misclassifications_fgsm = []
misclassifications_pgd = []
FLAGS = flags.FLAGS
print('Resnet result')
print()
for image_index in range(total_images):
  
  print('-------------------------------')
  print()
  print('Image index :',image_index)
  print()
  print('Original Sample')
  image  = X_test[image_index]
  image = img_to_array(image)
  image = image.reshape(1, 32, 32, 3)
  true_value = y_test[image_index]
  original_prediction = make_predictions(resnet,image,true_value)
  fgsm_sample = fast_gradient_method(logits_resnet, image, epsilon, np.inf, targeted=False)
  pgd_sample = projected_gradient_descent(resnet, image, 0.2, 0.01, 40, np.inf)
  print()
  print('Untargeted FGSM Adverserial Sample')
  fgsm_prediction = make_predictions(resnet , fgsm_sample , true_value)
  print()
  print('PGD Adverserial Sample')
  pgd_prediction = make_predictions(resnet , pgd_sample , true_value)
  print()
  
  if(pgd_prediction[0]!=original_prediction[0]):
    original_model_label = original_prediction[0]
    pgd_label = pgd_prediction[0]
    original_probability = original_prediction[1]
    pgd_probability = pgd_prediction[1]
    misclassifications_pgd.append({'Original Model Label':original_model_label,'original probability':original_probability,'Prediction after PGD attack':pgd_label,'Probability after PGD attack':pgd_probability,'Image index':image_index})

  if(fgsm_prediction[0]!=original_prediction[0]):
    original_model_label = original_prediction[0]
    fgsm_label = fgsm_prediction[0]
    original_probability = original_prediction[1]
    fgsm_probability = fgsm_prediction[1]
    misclassifications_fgsm.append({'Original Model Label':original_model_label,'original probability':original_probability,'Prediction after FGSM attack':fgsm_label,'Probability after FGSM attack':fgsm_probability,'Image index':image_index})
     
  print('-------------------------------')


Streaming output truncated to the last 5000 lines.



PGD Adverserial Sample


Predicted Label - airplane  Probability - 0.99996746
True Label - airplane



-------------------------------
-------------------------------

Image index : 28

Original Sample


Predicted Label - truck  Probability - 0.99999976
True Label - truck



Untargeted FGSM Adverserial Sample


Predicted Label - truck  Probability - 0.9999995
True Label - truck



PGD Adverserial Sample


Predicted Label - truck  Probability - 0.9999995
True Label - truck



-------------------------------
-------------------------------

Image index : 29

Original Sample


Predicted Label - frog  Probability - 1.0
True Label - frog



Untargeted FGSM Adverserial Sample


Predicted Label - frog  Probability - 1.0
True Label - frog



PGD Adverserial Sample


Predicted Label - frog  Probability - 1.0
True Label - frog



-------------------------------
-------------------------------

Image index : 30

Original Sample


Predicted Lab

In [21]:
print('Mispredictions due to FGSM attack')
print()
print('-------------------------------')
counter = 0
for element in misclassifications_fgsm:
  print(element)
  counter+=1
print()
print('Total count - ',counter)

Mispredictions due to FGSM attack

-------------------------------
{'Original Model Label': 'deer', 'original probability': 0.5534122, 'Prediction after FGSM attack': 'frog', 'Probability after FGSM attack': 0.92280513, 'Image index': 4}
{'Original Model Label': 'airplane', 'original probability': 0.7332395, 'Prediction after FGSM attack': 'dog', 'Probability after FGSM attack': 0.741324, 'Image index': 10}
{'Original Model Label': 'deer', 'original probability': 0.91956365, 'Prediction after FGSM attack': 'ship', 'Probability after FGSM attack': 0.5610923, 'Image index': 32}
{'Original Model Label': 'deer', 'original probability': 0.5783623, 'Prediction after FGSM attack': 'horse', 'Probability after FGSM attack': 0.8249995, 'Image index': 36}
{'Original Model Label': 'dog', 'original probability': 0.8297396, 'Prediction after FGSM attack': 'cat', 'Probability after FGSM attack': 0.5448027, 'Image index': 57}
{'Original Model Label': 'cat', 'original probability': 0.73968834, 'Predict

In [22]:
print('Mispredictions due to PGD attack')
print()
print('-------------------------------')
counter = 0
for element in misclassifications_pgd:
  print(element)
  counter+=1
print()
print(counter)

Mispredictions due to PGD attack

-------------------------------
{'Original Model Label': 'deer', 'original probability': 0.5534122, 'Prediction after PGD attack': 'frog', 'Probability after PGD attack': 0.926677, 'Image index': 4}
{'Original Model Label': 'airplane', 'original probability': 0.7332395, 'Prediction after PGD attack': 'dog', 'Probability after PGD attack': 0.7560828, 'Image index': 10}
{'Original Model Label': 'deer', 'original probability': 0.91956365, 'Prediction after PGD attack': 'ship', 'Probability after PGD attack': 0.6526779, 'Image index': 32}
{'Original Model Label': 'deer', 'original probability': 0.5783623, 'Prediction after PGD attack': 'horse', 'Probability after PGD attack': 0.83790416, 'Image index': 36}
{'Original Model Label': 'dog', 'original probability': 0.8297396, 'Prediction after PGD attack': 'cat', 'Probability after PGD attack': 0.5557563, 'Image index': 57}
{'Original Model Label': 'cat', 'original probability': 0.73968834, 'Prediction after P